In [ ]:
# Cell 1: Installation
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 57.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.49-py3-none-any.whl size=23742426 sha256=3eecd8cd29cd6dfb7871450e0de072d7a139e99a956baf9b26ac99c4905955fc
  Stored in directory: /root/.cache/pip/wheels/43/fe/dc/a0de3c289cfd5923ece6524469d328950e14fa0c90b1088ffa
Successfully built owlready2


In [ ]:
from owlready2 import *

# We define namespaces
dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul_ns = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
onto = get_ontology("http://www.semanticweb.org/gamematcher.owl")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Andreea Scrob, Edoardo Tommasi")
    onto.metadata.comment.append("GameMatcher is an intelligent companion system that assists users in finding video games compatible with their computer.")
    onto.metadata.label.append("GameMatcher Ontology")

    # TAXONOMY (CLASSES)
    #  USER
    class User(Thing):
        comment = ["An Agent who interacts with the system to find compatible video games."]
        label = ["User"]
        is_a = [dul_ns.Agent]

    # COMPUTER and COMPONENTS
    # Note: A Computer is a Physical Object, not a subclass of Component.
    class Computer(Thing):
        comment = ["A Physical Object consisting of hardware components capable of executing software."]
        label = ["Computer"]
        is_a = [dul_ns.PhysicalObject]

    class Component(Thing):
        comment = ["A Physical Object that forms a constituent part of a computing system."]
        label = ["Component"]
        is_a = [dul_ns.PhysicalObject]

    # Specific Hardware Classes
    class CPU(Component):
        comment = ["A Hardware Component that acts as the primary processing unit."]
        label = ["CPU"]

    class GPU(Component):
        comment = ["A Hardware Component specialized for rendering graphics."]
        label = ["GPU"]

    class RAM(Component):
        comment = ["A Hardware Component used for temporary data storage."]
        label = ["RAM"]

    class Storage(Component):#da specificare nella relazione che questo diventa Storage poi lo spazio disponibile viene preso con la dataproperty
        comment = ["A Hardware Component used for data retention (e.g., HDD or SSD)."]
        label = ["Storage"]

    # Specific Software Classes
    class OperatingSystem(Thing):
        comment = ["A Software Component that manages computer hardware and resources."]
        label = ["Operating System"]
        is_a = [dul_ns.InformationObject]

    #  VIDEOGAME and REQUIREMENTS
    class VideoGame(Thing):
        comment = ["An Information Object representing a digital game available for play."]
        label = ["Video Game"]
        is_a = [dul_ns.InformationObject]

    class MinimumRequirement(Thing):
        comment = ["A Requirement specifying the absolute minimum specifications for a game to run."]
        label = ["Minimum Requirement"]
        is_a = [dul_ns.Description]


    #  ATTRIBUTES and CONCEPTS
    #todo: chiedere al prof se possiamo aggiungere GameAttribute
    class GameAttribute(Thing):
        comment = ["A Concept representing a characteristic feature of a video game."]
        label = ["Game Attribute"]
        is_a = [dul_ns.Concept]

    class GameGenre(GameAttribute):
        comment = ["A classification based on gameplay interaction (e.g., RPG, Strategy)."]
        label = ["Game Genre"]

    class PlayerMode(GameAttribute):
        comment = ["Defines the number of players supported (e.g., Single-player)."]
        label = ["Player Mode"]

    class PEGIRating(GameAttribute):
        comment = ["Indicates the suitability of content for different age groups."]
        label = ["PEGI Rating"]

    # 1. Top-Level Disjointness
    # A User is not a Computer, which is not a Game, etc.
    AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])

    # 2. Hardware Component Disjointness
    # A CPU is not a GPU, RAM, or Storage.
    AllDisjoint([CPU, GPU, RAM, Storage])

    # 3. Attribute Disjointness
    # A Genre is not a PEGI Rating or a Player Mode.
    AllDisjoint([GameGenre, PlayerMode, PEGIRating])


    # OBJECT PROPERTIES (RELATIONS)
    # Relations: Computer -> Components
    class hasComponent(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]
        range = [Component]
        comment = ["Generic relation linking a computer to its parts."]
        label = ["has component"]


    # Sub-properties for specificity
    class hasCPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [CPU]
        comment = ["Relation linking components to the computer's cpu."]
        label = ["has CPU"]

    class hasGPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [GPU]
        comment = ["Relation linking components to the computer's gpu."]
        label = ["has GPU"]

    class hasRAM(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [RAM]
        comment = ["Relation linking components to the computer's ram."]
        label = ["has RAM"]

    class hasStorage(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [Storage]
        comment = ["Relation linking components to the computer's  available storage."]
        label = ["has storage"]

    class hasOS(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [OperatingSystem]
        comment = ["Relation linking components to the computer's os."]
        label = ["has operating system"]


    # Relations: Videogame -> Requirements
    class hasMinRequirement(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [MinimumRequirement]
        comment = ["Relation linking videogame to its component requirements to make it run."]
        label = ["has minimum requirement"]


    # Relations: User -> Computer
    class hasComputer(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]
        range = [Computer]
        comment = ["Relation linking user to the computer he owns."]
        label = ["has computer device"]



    # Relations: Attribute
    class hasGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [GameGenre]
        comment = ["Relation linking videogame to its genre."]
        label = ["has genre"]


    class preferredGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [GameGenre]
        comment = ["Relation linking user to its preferred game's genre."]
        label = ["prefers genre"]

    class hasPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PlayerMode]
        comment = ["Relation linking videogame to its player mode."]
        label = ["has player mode"]


    class preferredPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [PlayerMode]
        comment = ["Relation linking user to its preferred game mode."]
        label = ["prefers player mode"]


    class hasPEGI(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PEGIRating]
        comment = ["Relation linking videogame to its age rating."]
        label = ["has PEGI rating"]


    # Derived Relation (for Compatibility Checking)
    class isCompatibleWith(ObjectProperty, SymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [Computer]
        comment = ["Inferred relation: true if the computer meets all game requirements."]
        label = ["is compatible with"]



    # DATA PROPERTIES (ACTIONABLE LOGIC)

    # BENCHMARK SCORE (CPU and GPU)
    # User Hardware Property
    class hasBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [CPU, GPU]
        range = [int]
        comment = ["The performance score of the user's hardware (e.g. PassMark)."]
        label = ["has benchmark score"]


    # Game Requirement Property
    class requiresBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        comment = ["The minimum benchmark score required by the game."]
        label = ["requires benchmark score"]


    #  MEMORY (RAM and VRAM)
    class hasMemorySizeGB(DataProperty, FunctionalProperty):
        domain = [RAM, GPU]
        range = [float]
        comment = ["The amount of memory in Gigabytes."]
        label = ["has memory size (GB)"]


    class requiresMemoryGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum RAM or VRAM required in Gigabytes."]
        label = ["requires memory (GB)"]


    #  STORAGE
    class hasStorageSizeGB(DataProperty, FunctionalProperty):
        domain = [Storage]
        range = [float]
        comment = ["Total available disk space in Gigabytes."]
        label = ["has storage size (GB)"]


    class requiresStorageSpaceGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Installation size required in Gigabytes."]
        label = ["requires storage space (GB)"]


    # OS LOGIC
    class hasOSVersionValue(DataProperty, FunctionalProperty):
        domain = [OperatingSystem]
        range = [float]
        comment = ["Numeric version of the OS (e.g., 10.0, 11.0)."]
        label = ["has OS version value"]


    class requiresMinOSVersionValue(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum numeric OS version required."]
        label = ["requires min OS version"]



   #  BUDGET and PRICING
    class hasPrice(DataProperty, FunctionalProperty):
        domain = [VideoGame]
        range = [float]
        comment = ["The monetary cost required to purchase the digital game."]
        label = ["has price"]


    class hasBudget(DataProperty, FunctionalProperty):
        domain = [User]
        range = [float]
        comment = ["The maximum monetary limit the user is willing to spend."]
        label = ["has budget"]


    #  AGE and PEGI
    class hasAge(DataProperty, FunctionalProperty):
        domain = [User]
        range = [int]
        comment = ["The chronological age of the user (in years), used for PEGI checks."]
        label = ["has age"]


    class hasPEGIAgeThreshold(DataProperty, FunctionalProperty):
        domain = [PEGIRating]
        range = [int]
        comment = ["The numeric age threshold (e.g., 18)."]
        label = ["has PEGI threshold"]




In [ ]:
# ==========================================
# TBOX REFINEMENTS (VINCOLI E REGOLE)
# ==========================================
# ATTENZIONE: Queste righe devono essere SENZA spazi all'inizio!

# 1. DISJOINTNESS
AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])
AllDisjoint([CPU, GPU, RAM, Storage])
AllDisjoint([GameGenre, PlayerMode, PEGIRating])

# 2. RESTRICTIONS
# Computer
Computer.is_a.append(hasCPU.exactly(1, CPU))
Computer.is_a.append(hasGPU.exactly(1, GPU))
Computer.is_a.append(hasRAM.min(1, RAM))
Computer.is_a.append(hasStorage.min(1, Storage))
Computer.is_a.append(hasOS.exactly(1, OperatingSystem))

# VideoGame
VideoGame.is_a.append(hasPrice.exactly(1, float))
VideoGame.is_a.append(hasPEGI.exactly(1, PEGIRating))
VideoGame.is_a.append(hasMinRequirement.min(1, MinimumRequirement))

# User
User.is_a.append(hasAge.exactly(1, int))
User.is_a.append(hasBudget.exactly(1, float))

print("Regole e Restrizioni applicate con successo!")

Regole e Restrizioni applicate con successo!


In [ ]:
# INSTANCES (ENTRIES)
# da rifare in base alle modifiche fatte

with onto:
    # 1. HARDWARE (Componenti Fisici)
    intel_i9 = CPU("Intel_Core_i9_13900K")
    intel_i9.hasBenchmarkScore = 60000

    nvidia_rtx = GPU("NVIDIA_GeForce_RTX_4090")
    nvidia_rtx.hasBenchmarkScore = 40000
    nvidia_rtx.hasMemorySizeGB = 24.0

    corsair_ram = RAM("Corsair_Vengeance_32GB")
    corsair_ram.hasMemorySizeGB = 32.0

    samsung_ssd = AvailableStorage("Samsung_990_Pro_2TB")
    samsung_ssd.hasStorageSizeGB = 2000.0

    # 2. SOFTWARE DI SISTEMA (OS)
    win11 = OperatingSystem("Windows_11_Home")
    win11.hasOSVersionValue = 11.0

    # 3. COMPUTER (Assemblaggio)
    my_gaming_pc = Computer("Super_Gaming_Rig")

    # Functional Properties (SENZA parentesi quadre [])
    my_gaming_pc.hasCPU = intel_i9
    my_gaming_pc.hasGPU = nvidia_rtx
    my_gaming_pc.hasOS = [win11]

    # Non-Functional Properties (CON parentesi quadre [])
    my_gaming_pc.hasRAM = [corsair_ram]
    my_gaming_pc.hasStorage = [samsung_ssd]

    # 4. DEFINIZIONI CATEGORICHE (Genre, PEGI, PlayerMode)
    rpg_genre = GameGenre("RPG")
    fps_genre = GameGenre("FPS")

    pegi_18 = PEGIRating("PEGI_18")
    pegi_18.hasPEGIAgeThreshold = 18

    mode_single = PlayerMode("Single_Player_Mode")

    # 5. VIDEOGIOCO & REQUISITI
    req_cyberpunk = MinimumRequirement("Cyberpunk2077_MinSpecs")
    req_cyberpunk.requiresBenchmarkScore = 15000
    req_cyberpunk.requiresMemoryGB = 16.0
    req_cyberpunk.requiresStorageSpaceGB = 70.0
    req_cyberpunk.requiresMinOSVersionValue = 10.0

    cyberpunk = VideoGame("Cyberpunk_2077")
    cyberpunk.hasPrice = 59.99
    cyberpunk.hasPlayerModeID = ["SinglePlayer"] # Data Property (Lista stringhe OK)

    # Functional Properties (SENZA parentesi quadre [])
    cyberpunk.hasMinRequirement = req_cyberpunk
    cyberpunk.hasPEGI = pegi_18

    # Non-Functional Properties (CON parentesi quadre [])
    cyberpunk.hasGenre = [rpg_genre]
    cyberpunk.hasPlayerMode = [mode_single]

    # 6. UTENTE (USER)
    john_doe = User("John_Doe")
    john_doe.hasAge = 25
    john_doe.hasBudget = 80.0
    john_doe.prefersPlayerModeID = ["SinglePlayer"]

    # Non-Functional Properties (CON parentesi quadre [])
    john_doe.hasComputer = [my_gaming_pc]
    john_doe.preferredGenre = [rpg_genre]
    john_doe.preferredPlayerMode = [mode_single]

print("Istanze create con successo!")

Istanze create con successo!


In [ ]:
# SAVE ONTOLOGY
output_file = "gameMatcher_Final.owl"
onto.save(file=output_file)

print(f"Success! Ontology saved to: {output_file}")
print("Features included: DUL Alignment, Singular naming, Actionable Data Properties, ABox Instances.")

Success! Ontology saved to: gameMatcher_Final.owl
Features included: DUL Alignment, Singular naming, Actionable Data Properties, ABox Instances.
